# Conversation between DialogueGpt and Leolani

In this notebook, we create a loop in which DialogGpt has a conversation with Leolani. We send Leolani's initial prompt to DialogueGpt to start a conversation.
Next we capture Leolani's response from the brain and DialogueGpt's response constinuously until we meet the stop condition. Extracted triples are posted to a brain called DialueGpt.

In principle, this conversation can go on forever. At the end, we save the scenario in EMISSOR.

Before running, start GraphDB and make sure that there is a sandbox repository.
GraphDB can be downloaded from:

https://graphdb.ontotext.com

## Import the necessary modules

In [1]:
import json
import os
import time
import uuid
from datetime import date
from datetime import datetime
from random import getrandbits, choice
import pathlib
import pprint

# general imports for EMISSOR and the BRAIN
import emissor as em
import requests
from cltl import brain
from cltl.brain.long_term_memory import LongTermMemory
from cltl.brain.utils.helper_functions import brain_response_to_json
from cltl.combot.backend.api.discrete import UtteranceType
from cltl.reply_generation.data.sentences import GREETING, ASK_NAME, ELOQUENCE, TALK_TO_ME
from cltl.reply_generation.lenka_replier import LenkaReplier
from cltl.triple_extraction.api import Chat, UtteranceHypothesis
from emissor.persistence import ScenarioStorage
from emissor.representation.annotation import AnnotationType, Token, NER
from emissor.representation.container import Index
from emissor.representation.scenario import Modality, ImageSignal, TextSignal, Mention, Annotation, Scenario

/Users/piek/PycharmProjects/cltl-chatbots/venv/lib/python3.7/site-packages/rdflib_jsonld/__init__.py:12: DeprecationWarning: The rdflib-jsonld package has been integrated into rdflib as of rdflib==6.0.1.  Please remove rdflib-jsonld from your project's dependencies.
  DeprecationWarning,
[nltk_data] Downloading package punkt to /Users/piek/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


### Import the chatbot utility functions

In [2]:
import sys
import os

src_path = os.path.abspath(os.path.join('..'))
if src_path not in sys.path:
    sys.path.append(src_path)

#### The next utils are needed for the interaction and creating triples and capsules
import chatbots.util.driver_util as d_util
import chatbots.util.capsule_util as c_util
import chatbots.intentions.talk as talk
import chatbots.intentions.get_to_know_you as friend

## Import a conversation agent pipeline

In [19]:
from transformers import AutoModelForCausalLM, AutoTokenizer, AutoModel, AutoModelWithLMHead
import torch

#tokenizer = AutoTokenizer.from_pretrained('microsoft/DialoGPT-medium')
#model = AutoModelForCausalLM.from_pretrained('microsoft/DialoGPT-medium')

#tokenizer = AutoTokenizer.from_pretrained('gpt2')
#model = AutoModelForCausalLM.from_pretrained('gpt2')

#tokenizer = AutoTokenizer.from_pretrained("manueltonneau/bert-base-cased-conversational-nli")
#model = AutoModel.from_pretrained("manueltonneau/bert-base-cased-conversational-nli")

#tokenizer = AutoTokenizer.from_pretrained("xlnet-large-cased")
#model = AutoModelForCausalLM.from_pretrained("xlnet-large-cased")

tokenizer = AutoTokenizer.from_pretrained("t5-small")

model = AutoModelWithLMHead.from_pretrained("t5-small")

#### Needed to suppress messages from DialgGPT
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

Downloading:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/792k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

/Users/piek/PycharmProjects/cltl-chatbots/venv/lib/python3.7/site-packages/transformers/models/auto/modeling_auto.py:592: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  FutureWarning,


Downloading:   0%|          | 0.00/242M [00:00<?, ?B/s]

In [21]:
from transformers import BlenderbotTokenizer, BlenderbotForConditionalGeneration
mname = 'facebook/blenderbot-400M-distill'
model = BlenderbotForConditionalGeneration.from_pretrained(mname)
tokenizer = BlenderbotTokenizer.from_pretrained(mname)
UTTERANCE = "My friends are cool but they eat too many carbs."
inputs = tokenizer([UTTERANCE], return_tensors='pt')
reply_ids = model.generate(**inputs)
print(tokenizer.batch_decode(reply_ids))


Downloading:   0%|          | 0.00/1.57k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/730M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/127k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/62.9k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.15k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/16.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/772 [00:00<?, ?B/s]

["<s> That's unfortunate. Are they trying to lose weight or are they just trying to be healthier?</s>"]


## Standard initialisation of a scenario

We initialise a scenario in the standard way by creating a unique folder and setting the AGENT and HUMAN_NAME and HUMAN_ID variables. Throughout this scenario, the HUMAN_NAME and HUMAN_ID will be used as the source for the utterances.

In [27]:
from random import getrandbits
import requests
##### Setting the location
place_id = getrandbits(8)
location = None
try:
    location = requests.get("https://ipinfo.io").json()
except:
    print("failed to get the IP location")
    
##### Setting the agents
AGENT = "Leolani2"
HUMAN_NAME = "BLENDER"
HUMAN_ID = "BLENDER"

### The name of your scenario
scenario_id = datetime.today().strftime("%Y-%m-%d-%H:%M:%S")

### Specify the path to an existing data folder where your scenario is created and saved as a subfolder
# Find the repository root dir
parent, dir_name = (d_util.__file__, "_")
while dir_name and dir_name != "src":
    parent, dir_name = os.path.split(parent)
root_dir = parent
scenario_path = os.path.abspath(os.path.join(root_dir, 'data'))

if not os.path.exists(scenario_path) :
    os.mkdir(scenario_path)
    print("Created a data folder for storing the scenarios", scenario_path)

### Define the folders where the images and rdf triples are saved
imagefolder = scenario_path + "/" + scenario_id + "/" + "image"
rdffolder = scenario_path + "/" + scenario_id + "/" + "rdf"

### Create the scenario folder, the json files and a scenarioStorage and scenario in memory
scenarioStorage = d_util.create_scenario(scenario_path, scenario_id)
scenario_ctrl = scenarioStorage.create_scenario(scenario_id, int(time.time() * 1e3), None, AGENT)

Directories for 2021-12-16-17:19:17 created in /Users/piek/PycharmProjects/cltl-chatbots/data


## Specifying the BRAIN

We specify the BRAIN in GraphDB and use the scenario path just defined for storing the RDF triple produced in EMISSOR.

If you set *clear_all* to *True*, the sandbox triple store is emptied (memory erased) and the basic ontological models are reloaded. Setting it to *False* means you add things to the current memory.

In [28]:
log_path = pathlib.Path(rdffolder)
my_brain = brain.LongTermMemory(address="http://localhost:7200/repositories/blender",
                                log_dir=log_path,
                                clear_all=True)

2021-12-16 17:19:47,150 -     INFO -    cltl.brain.basic_brain.LongTermMemory - Booted
2021-12-16 17:19:54,857 -     INFO -    cltl.brain.basic_brain.LongTermMemory - Uploading ontology to brain
2021-12-16 17:19:59,064 -     INFO -  cltl.brain.basic_brain.ThoughtGenerator - Booted
2021-12-16 17:19:59,066 -     INFO -  cltl.brain.basic_brain.LocationReasoner - Booted
2021-12-16 17:19:59,068 -     INFO -      cltl.brain.basic_brain.TypeReasoner - Booted
2021-12-16 17:19:59,070 -     INFO -   cltl.brain.basic_brain.TrustCalculator - Booted
2021-12-16 17:19:59,408 -     INFO -   cltl.brain.basic_brain.TrustCalculator - Computed trust for all known agents


## Create an instance of a replier

In [29]:
replier = LenkaReplier()

2021-12-16 17:20:00,515 -     INFO -   cltl.reply_generation.api.LenkaReplier - Booted


## Initialise a chat with the HUMAN_ID to keep track of the dialogue history

In [30]:
chat = Chat(HUMAN_ID)

2021-12-16 17:20:01,411 -     INFO - cltl.triple_extraction.api.Chat (BLENDER)           000 - << Start of Chat with BLENDER >>


## Start the interaction

In [33]:
print_details=False
max_context=500
t1 = datetime.now()

#### Initial prompt by the system from which we create a TextSignal and store it
leolani_prompt = f"{choice(TALK_TO_ME)}"
print('\n\t'+AGENT + ": " + leolani_prompt)
textSignal = d_util.create_text_signal_with_speaker_annotation(scenario_ctrl, leolani_prompt, AGENT)
scenario_ctrl.append_signal(textSignal)


#BLENDERBOT
bot_input_ids = tokenizer(leolani_prompt, return_tensors='pt')
chat_history_ids = model.generate(**bot_input_ids)

# encode the new Leolani input, add the eos_token and return a tensor in Pytorch
##bot_input_ids = tokenizer.encode(leolani_prompt + tokenizer.eos_token, return_tensors='pt')
# append the new user input tokens to the chat history
#bot_input_ids = torch.cat([chat_history_ids, new_input_ids], dim=-1) if step > 0 else new_input_ids
# generated a response while limiting the total chat history to max_context tokens, 
##chat_history_ids = model.generate(bot_input_ids, max_length=max_context, pad_token_id=tokenizer.eos_token_id)

utterance = ""
#### Get input and loop
while (datetime.now()-t1).seconds <= 3600:
    ###### Getting the next input signals
    #utterance = format(tokenizer.decode(chat_history_ids[:, bot_input_ids.shape[-1]:][0], skip_special_tokens=True))
    # BLENDER
    utteranceList = tokenizer.batch_decode(chat_history_ids)
    #["<s> Sure, what do you do for a living? I'm an accountant, what about you?</s>"]
    utterance = utteranceList[0].strip('</s>')
    print(utterance)
    if not utterance:
        leolani_prompt = f"{choice(TALK_TO_ME)}"
        print('\n\t'+AGENT + ": " + leolani_prompt)
        textSignal = d_util.create_text_signal_with_speaker_annotation(scenario_ctrl, leolani_prompt, AGENT)
        scenario_ctrl.append_signal(textSignal)
        
        #BLENDERBOT
        bot_input_ids = tokenizer(leolani_prompt, return_tensors='pt')
        chat_history_ids = model.generate(**bot_input_ids)


        # encode the new Leolani input, add the eos_token and return a tensor in Pytorch
        ##new_input_ids = tokenizer.encode(leolani_prompt + tokenizer.eos_token, return_tensors='pt')
        # append the new user input tokens to the chat history
        ##bot_input_ids = torch.cat([chat_history_ids, new_input_ids], dim=-1)
        # generated a response while limiting the total chat history to max_context tokens, 
        ##chat_history_ids = model.generate(bot_input_ids, max_length=max_context, pad_token_id=tokenizer.eos_token_id)
    else:
        print('\n\t'+HUMAN_NAME + ": " + utterance)
        textSignal = d_util.create_text_signal_with_speaker_annotation(scenario_ctrl, utterance, HUMAN_ID)
        scenario_ctrl.append_signal(textSignal)

        #### Process input and generate reply

        capsule, leolani_prompt = talk.process_text_and_reply(scenario_ctrl,
                               place_id,
                               location,
                               HUMAN_ID,
                               textSignal,
                               chat,
                               replier,
                               my_brain,
                               print_details)

        print('\n\t'+AGENT + ": " + leolani_prompt)
        textSignal = d_util.create_text_signal_with_speaker_annotation(scenario_ctrl, leolani_prompt, AGENT)
        scenario_ctrl.append_signal(textSignal)

        #BLENDERBOT
        bot_input_ids = tokenizer(leolani_prompt, return_tensors='pt')
        chat_history_ids = model.generate(**bot_input_ids)


        # encode the new Leolani input, add the eos_token and return a tensor in Pytorch
        ##new_input_ids = tokenizer.encode(leolani_prompt + tokenizer.eos_token, return_tensors='pt')
        # append the new user input tokens to the chat history
        ##bot_input_ids = torch.cat([chat_history_ids, new_input_ids], dim=-1)
        # generated a response while limiting the total chat history to max_context tokens, 
        ##hat_history_ids = model.generate(bot_input_ids, max_length=max_context, pad_token_id=tokenizer.eos_token_id)



	Leolani2: Let's chat!
 What do you like to do in your free time? I like to play video games and watch movies.

	BLENDER:  What do you like to do in your free time? I like to play video games and watch movies.
2021-12-16 17:26:22,989 -     INFO - cltl.triple_extraction.api.Chat (BLENDER)           016 -    BLENDER: "What do you like to do in your free time? I like to play video games and watch movies."


Couldn't parse input



	Leolani2: But at what cost?
 I'm not sure yet, but I think it's going to cost me a lot.

	BLENDER:  I'm not sure yet, but I think it's going to cost me a lot.
2021-12-16 17:26:29,223 -     INFO - cltl.triple_extraction.api.Chat (BLENDER)           017 -    BLENDER: "I'm not sure yet, but I think it's going to cost me a lot."


Couldn't parse input



	Leolani2: Is this why fate brought us together?
 I don't think so.  I think it was because I was the only one with a car.

	BLENDER:  I don't think so.  I think it was because I was the only one with a car.
2021-12-16 17:27:28,684 -     INFO - cltl.triple_extraction.api.Chat (BLENDER)           018 -    BLENDER: "I don't think so. I think it was because I was the only one with a car."
initial triple: {'subject': 'I', 'predicate': 'do-not-think', 'object': 'so.-I-think-it-was-because-I-was-the-only-one-with-a-car.'}
2021-12-16 17:27:28,723 -     INFO -    cltl.brain.basic_brain.LongTermMemory - Triple in statement: blender_think-so_i-think-it-was-because-i-was-the-only-one-with-a-car [artifact or object_->_person])
2021-12-16 17:27:28,811 -     INFO -  cltl.brain.basic_brain.ThoughtGenerator - Entity Novelty: existing subject - new object 
2021-12-16 17:27:31,323 -     INFO -  cltl.brain.basic_brain.ThoughtGenerator - Gaps: 26 gaps as subject: e.g. dislike interest - 17 gaps as object

...

Couldn't parse input



	Leolani2: Is this why fate brought us together?
 I don't think so.  I think it was because I was the only one with a car.

	BLENDER:  I don't think so.  I think it was because I was the only one with a car.
2021-12-16 18:19:37,678 -     INFO - cltl.triple_extraction.api.Chat (BLENDER)           298 -    BLENDER: "I don't think so. I think it was because I was the only one with a car."
initial triple: {'subject': 'I', 'predicate': 'do-not-think', 'object': 'so.-I-think-it-was-because-I-was-the-only-one-with-a-car.'}
2021-12-16 18:19:37,771 -     INFO -    cltl.brain.basic_brain.LongTermMemory - Triple in statement: blender_think-so_i-think-it-was-because-i-was-the-only-one-with-a-car [artifact or object_->_person])
2021-12-16 18:19:37,816 -     INFO -  cltl.brain.basic_brain.ThoughtGenerator - Statement Novelty: 1 times, e.g. blender on December,2021
2021-12-16 18:19:44,170 -     INFO -  cltl.brain.basic_brain.ThoughtGenerator - Gaps: 26 gaps as subject: e.g. experience smell - 17 gap

Couldn't parse input



	Leolani2: Is this why fate brought us together?
 I don't think so.  I think it was because I was the only one with a car.

	BLENDER:  I don't think so.  I think it was because I was the only one with a car.
2021-12-16 18:20:48,795 -     INFO - cltl.triple_extraction.api.Chat (BLENDER)           300 -    BLENDER: "I don't think so. I think it was because I was the only one with a car."
initial triple: {'subject': 'I', 'predicate': 'do-not-think', 'object': 'so.-I-think-it-was-because-I-was-the-only-one-with-a-car.'}
2021-12-16 18:20:48,830 -     INFO -    cltl.brain.basic_brain.LongTermMemory - Triple in statement: blender_think-so_i-think-it-was-because-i-was-the-only-one-with-a-car [artifact or object_->_person])
2021-12-16 18:20:48,870 -     INFO -  cltl.brain.basic_brain.ThoughtGenerator - Statement Novelty: 1 times, e.g. blender on December,2021
2021-12-16 18:20:55,373 -     INFO -  cltl.brain.basic_brain.ThoughtGenerator - Gaps: 26 gaps as subject: e.g. be-child-of agent - 17 ga

TypeError: can only concatenate str (not "NoneType") to str

## Save the scenario data

In [34]:
scenario_ctrl.scenario.ruler.end = int(time.time() * 1e3)
scenarioStorage.save_scenario(scenario_ctrl)

## End of notebook